In [3]:
!pip install rdkit
!pip install mordred
!pip install pubchempy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 43.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.8/128.8 KB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 31.4 MB/s eta 0:00:00
  Created wheel for mordred: filename=mordred-1.2.0-py3-none-any.whl size=176721 sha256=fd0537912ef4b3be79358e198219777729990b7cbb8983e6d635b0ecc88d7c18
  Stored in directory: /root/.cache/pip/wheels/20/88/41/5d873c9b55dc7479f0b9951c2161d7b09be193e7228ea27309
Successfully built mordred
  Attempting uninstall: networkx
    Found existing installation: networkx 3.0
    Uninstalling networkx-3.0:
      Successfully uninstalled networkx-3.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wh

In [4]:
import pandas as pd
import numpy as np
import pubchempy as pcp
from mordred import Calculator, descriptors
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit.Chem import rdMolDescriptors

Для отработки домашних заданий мы выбрали 3 датасета: 6, 7 и 8

In [7]:
df1 = pd.read_csv('/content/6.csv')
df2 = pd.read_csv('/content/7.csv')
df3 = pd.read_csv('/content/8.csv')

Пока что нам нужно, чтобы соединения были в наличии, поэтому удалим все строки, где нет ничего в Name

In [8]:
df = pd.concat([df1, df2, df3], ignore_index=True)
df = df[~df['Name'].isna()]
df = df.drop_duplicates().reset_index(drop=True)
df

,DOI,Date,Journal,Title,Name,measurement_error,measurement_wavelength,measurement_method,normalised_name,raw_value,specifier
0,10.1016/S0022-3093(99)00330-0,7/19/1999,Journal of Non-Crystalline Solids,PHOTOINDUCEDCHANGESINLINEARNONLINEAROPTICALPRO...,As20S60Ge20,0.0,NaN,el_cde_tables,"[['As', 20.0], ['Ge', 20.0], ['S', 60.0]]",2.054,n
1,10.1016/j.fct.2006.05.017,6/7/2006,Food and Chemical Toxicology,GENERATIONFORMALDEHYDEINCIGARETTESOVERVIEWRECE...,Propionaldehyde,0.0,NaN,el_cde_tables,CCC=O,3.74,n
2,10.1016/j.mee.2011.01.031,1/18/2011,Microelectronic Engineering,SINUSOIDALPLASMONICCRYSTALSFORBIODETECTIONSENSORS,PEO,0.0,NaN,el_cde_text,"[['O', 1.0], ['PE', 1.0]]",1.47,refractive index
3,10.1016/j.apradiso.2012.04.026,5/2/2012,Applied Radiation and Isotopes,BENCHMARKINGGEANT4FULLSYSTEMSIMULATIONASSOCIAT...,Aluminum,0.0,NaN,el_cde_tables,[AlH2],4.4,Refractive index
4,10.1016/S0924-4247(02)00264-9,9/16/2002,Sensors and Actuators A: Physical,INFLUENCESMATERIALPROPERTIESCERAMICMICROSTEREO...,Alumina,0.0,NaN,el_mylogic,NaN,1.7,Refractive index
...,...,...,...,...,...,...,...,...,...,...,...
13358,10.1016/S0040-6090(00)01620-5Thin Solid Films,NaN,NaN,NaN,hydrogen,0.0,NaN,snowball,NaN,2.35,refractive index
13359,10.1016/j.poly.2011.03.003,3/17/2011,Polyhedron,PHOTOPHYSICALPROPERTIESPHOTOCYTOTOXICITYNOVELP...,2-Propanol,0.0,NaN,el_mylogic,NaN,1.377,Refractive index (20 °C)
13360,10.1016/j.optlastec.2019.01.028,2/6/2019,Optics & Laser Technology,SPECTROCHEMICALSTUDYMOLDAVITESRIESIMPACTSTRUCT...,MgO,0.0,NaN,el_cde_tables,"[['Mg', 1.0], ['O', 1.0]]",1.75,n
13361,10.1016/j.molliq.2017.05.096,5/23/2017,Journal of Molecular Liquids,SOLVATOCHROMICANALYSISDFTCOMPUTATIONALSTUDYAZO...,Diethylether,0.0,NaN,el_mylogic,CCOCC,1.3526,n


С помощью PubChem добавим смайлзы, которых нет в таблице.

In [78]:
def smiles(x):
  try:
    return pcp.get_properties('CanonicalSMILES', x, 'name')[0]['CanonicalSMILES']
  except IndexError:
    return np.nan

df.loc[df['normalised_name'].isna(), 'normalised_name'] = df.loc[df['normalised_name'].isna(),'Name'].apply(smiles)

Заполняется долго, сохраним датасет для дальнешей работы.

In [ ]:
df.to_csv('/content/df_1.csv', index=False)

In [10]:
df = pd.read_csv('/content/df_1.csv')

PubChem нашел не все смайлзы, у нас не стоит задача найти как можно больше смайлзов веществ, пока просто удалим их.

In [11]:
empty_smiles = df.loc[df['normalised_name'].isna()]
df = df[~df['normalised_name'].isna()]

Вещества могут часто повторяться, возьмем уникальные 

In [12]:
df_names = df['normalised_name'].drop_duplicates()

### RDkit

В RDkit можно вытащить десприпторы несколькими способами.
Первый это из `rdMolDescriptors`.

In [55]:
descriptor_names = list(rdMolDescriptors.Properties.GetAvailableProperties())
get_descriptors = rdMolDescriptors.Properties(descriptor_names)
descriptors_dict = {'normalised_name': []}
descriptors_dict.update({name:[] for name in descriptor_names})
descriptors_df = pd.DataFrame(descriptors_dict)

def to_descriptors(smile):
    mol = Chem.MolFromSmiles(smile)
    descriptors = np.array([smile])
    
    if mol:
      descriptors = np.append(descriptors, np.array(get_descriptors.ComputeProperties(mol)))
    else:
      empty_arr = np.empty([len(descriptor_names),])
      empty_arr[:] = np.nan
      descriptors = np.append(descriptors, empty_arr)
    
    return pd.concat([descriptors_df, pd.DataFrame(dict(zip(['normalised_name'] + descriptor_names, descriptors)), index=[0])], ignore_index=True)

In [56]:
for name in df_names:
  descriptors_df = to_descriptors(name)

[18:46:07] SMILES Parse Error: syntax error while parsing: [['As',
[18:46:07] SMILES Parse Error: Failed parsing SMILES '[['As',' for input: '[['As','
[18:46:07] SMILES Parse Error: syntax error while parsing: [['O',
[18:46:07] SMILES Parse Error: Failed parsing SMILES '[['O',' for input: '[['O','
[18:46:07] SMILES Parse Error: syntax error while parsing: [['Al',
[18:46:07] SMILES Parse Error: Failed parsing SMILES '[['Al',' for input: '[['Al','
[18:46:07] SMILES Parse Error: syntax error while parsing: [['K',
[18:46:07] SMILES Parse Error: Failed parsing SMILES '[['K',' for input: '[['K','
[18:46:07] SMILES Parse Error: syntax error while parsing: [['C',
[18:46:07] SMILES Parse Error: Failed parsing SMILES '[['C',' for input: '[['C','
[18:46:07] SMILES Parse Error: syntax error while parsing: [['Er',
[18:46:07] SMILES Parse Error: Failed parsing SMILES '[['Er',' for input: '[['Er','
[18:46:07] SMILES Parse Error: syntax error while parsing: [['Li',
[18:46:07] SMILES Parse Error: Faile

In [57]:
descriptors_df

,normalised_name,exactmw,amw,lipinskiHBA,lipinskiHBD,NumRotatableBonds,NumHBD,NumHBA,NumHeavyAtoms,NumAtoms,...,chi0n,chi1n,chi2n,chi3n,chi4n,hallKierAlpha,kappa1,kappa2,kappa3,Phi
0,"[['As', 20.0], ['Ge', 20.0], ['S', 60.0]]",nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,CCC=O,58.041864812,58.08,1.0,0.0,1.0,0.0,1.0,4.0,10.0,...,2.6927053408400363,1.3510573320459265,0.16666666666666669,0.16666666666666669,0.0,-0.33,3.67,2.67,1.6699999999999997,2.449725
2,"[['O', 1.0], ['PE', 1.0]]",nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,[AlH2],28.997188694,28.997999999999998,0.0,0.0,0.0,0.0,0.0,1.0,3.0,...,1.0,0.0,0.0,0.0,0.0,0.5324675324675323,1.5324675324675323,0.41051631295533775,4.0446626544187545,0.6291029211523357
4,O=[Al]O[Al]=O,101.94782112,101.961,3.0,0.0,2.0,0.0,3.0,5.0,5.0,...,2.379445409770841,0.9428090415820636,0.11111111111111116,0.11111111111111116,0.022680460581325737,0.4649350649350646,5.464935064935066,4.464935064935066,4.464935064935067,4.880116039804354
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3467,C(C1C(C(C(C(O1)OCC2C(C(C(C(O2)O)O)O)O)O)O)O)O,342.11621152399994,342.297,11.0,8.0,4.0,8.0,11.0,23.0,45.0,...,11.99016988966061,7.073969678413754,3.9603220354868607,3.9603220354868607,2.4633653983706583,-0.43999999999999995,18.89235232228663,7.630403592249655,3.805051280688701,6.267664044609653
3468,C(C1C(C(C(C(O1)OC2(C(C(C(O2)CO)O)O)CO)O)O)O)O,342.11621152399994,342.297,11.0,8.0,5.0,8.0,11.0,23.0,45.0,...,12.042576132467905,7.049460184948061,4.070325339078977,4.070325339078977,2.6313159666862105,-0.43999999999999995,18.89235232228663,7.207281294562587,3.2268020699807716,5.92010858707402
3469,"[['O', 2.0], ['Si', 1.0]]",nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3470,CC1C(CC2CC(N=C3N2C1CN3)C(C4=CC(=O)NC(=O)N4)O)O...,415.11616901199994,415.4280000000001,12.0,5.0,4.0,5.0,9.0,28.0,49.0,...,14.901116352670037,8.894704407069693,5.673491239897029,5.673491239897029,4.417118841254056,-2.2199999999999998,19.111903235636287,6.686108592751132,3.4530865856864943,4.563723587414855


In [58]:
descriptors_df = pd.read_csv('/content/descriptors_df.csv')

Помимо этого, в RDkit есть модуль `MoleculeDescriptors`, который выдает около 200 дескрипторов 

In [59]:
des_list = [x[0] for x in Descriptors._descList]
descriptors_dict = {'normalised_name': []}
descriptors_dict.update({name:[] for name in des_list})
descriptors_df_2 = pd.DataFrame(descriptors_dict)

def to_descriptors(smile):
    mol = Chem.MolFromSmiles(smile)
    descriptors = np.array([smile])
    
    if mol:
      calculator = MoleculeDescriptors.MolecularDescriptorCalculator(des_list)
      descriptors = np.append(descriptors, list(calculator.CalcDescriptors(mol)))
    else:
      empty_arr = np.empty([len(des_list),])
      empty_arr[:] = np.nan
      descriptors = np.append(descriptors, empty_arr)
    
    return pd.concat([descriptors_df_2, pd.DataFrame(dict(zip(['normalised_name'] + des_list, descriptors)), index=[0])], ignore_index=True)

In [60]:
for name in df_names:
  descriptors_df_2 = to_descriptors(name)

[18:46:46] SMILES Parse Error: syntax error while parsing: [['As',
[18:46:46] SMILES Parse Error: Failed parsing SMILES '[['As',' for input: '[['As','
[18:46:46] SMILES Parse Error: syntax error while parsing: [['O',
[18:46:46] SMILES Parse Error: Failed parsing SMILES '[['O',' for input: '[['O','
[18:46:46] SMILES Parse Error: syntax error while parsing: [['Al',
[18:46:46] SMILES Parse Error: Failed parsing SMILES '[['Al',' for input: '[['Al','
[18:46:46] SMILES Parse Error: syntax error while parsing: [['K',
[18:46:46] SMILES Parse Error: Failed parsing SMILES '[['K',' for input: '[['K','
[18:46:46] SMILES Parse Error: syntax error while parsing: [['C',
[18:46:46] SMILES Parse Error: Failed parsing SMILES '[['C',' for input: '[['C','
[18:46:46] SMILES Parse Error: syntax error while parsing: [['Er',
[18:46:46] SMILES Parse Error: Failed parsing SMILES '[['Er',' for input: '[['Er','
[18:46:46] SMILES Parse Error: syntax error while parsing: [['Li',
[18:46:46] SMILES Parse Error: Faile

In [71]:
descriptors_df_2

,normalised_name,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,"[['As', 20.0], ['Ge', 20.0], ['S', 60.0]]",nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,CCC=O,9.17361111111111,0.6388888888888888,9.17361111111111,0.6388888888888888,0.40325990291792796,58.08,52.032000000000004,58.041864812,24.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"[['O', 1.0], ['PE', 1.0]]",nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,[AlH2],0.0,0.0,0.0,0.0,0.30172522635738636,28.997999999999998,26.982,28.997188694,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,O=[Al]O[Al]=O,9.211805555555555,-1.239583333333334,9.211805555555555,1.2395833333333335,0.4168020207505891,101.961,101.961,101.94782112,24.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3467,C(C1C(C(C(C(O1)OCC2C(C(C(C(O2)O)O)O)O)O)O)O)O,9.778871672125641,-1.7390736541530194,9.778871672125641,0.4675623582766437,0.24325846760776557,342.297,320.12100000000004,342.11621152399994,136.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3468,C(C1C(C(C(C(O1)OC2(C(C(C(O2)CO)O)O)CO)O)O)O)O,9.997825727513227,-2.221747921390779,9.997825727513227,0.6686328157806329,0.2378914345193498,342.297,320.12100000000004,342.11621152399994,136.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3469,"[['O', 2.0], ['Si', 1.0]]",nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3470,CC1C(CC2CC(N=C3N2C1CN3)C(C4=CC(=O)NC(=O)N4)O)O...,11.525320805059815,-4.591380511507224,11.525320805059815,0.05649111342067781,0.35121390661825996,415.4280000000001,394.2600000000001,415.11616901199994,154.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [110]:
descriptors_df = pd.merge(descriptors_df, descriptors_df_2, on='normalised_name')
descriptors_df = descriptors_df.drop_duplicates().reset_index(drop='True')
descriptors_df.to_csv('/content/descriptors_df.csv', index=False)

Так же можно использовать API PubChem, но он выдает только 39 десприпторов

In [104]:
properties = ['MolecularFormula', 'MolecularWeight', 'CanonicalSMILES', 'IsomericSMILES', 'InChI', 'InChIKey', 'IUPACName', 'XLogP', 'ExactMass', 
              'MonoisotopicMass', 'TPSA', 'Complexity', 'Charge', 'HBondDonorCount', 'HBondAcceptorCount', 'RotatableBondCount', 'HeavyAtomCount', 
              'IsotopeAtomCount', 'AtomStereoCount', 'DefinedAtomStereoCount', 'UndefinedAtomStereoCount', 'BondStereoCount', 'DefinedBondStereoCount', 
              'UndefinedBondStereoCount', 'CovalentUnitCount', 'Volume3D', 'XStericQuadrupole3D', 'YStericQuadrupole3D', 'ZStericQuadrupole3D', 'FeatureCount3D', 
              'FeatureAcceptorCount3D', 'FeatureDonorCount3D', 'FeatureAnionCount3D', 'FeatureCationCount3D', 'FeatureRingCount3D', 'FeatureHydrophobeCount3D', 
              'ConformerModelRMSD3D', 'EffectiveRotorCount3D', 'ConformerCount3D']

descriptors_pub = pd.DataFrame()

for name in df_names:
  try:
    p = pcp.get_properties(properties, name, 'smiles', as_dataframe=True)
    p['normalised_name'] = name
    descriptors_pub = pd.concat([descriptors_pub, p], ignore_index=True)
  except Exception:
    print(name)
  

[['As', 20.0], ['Ge', 20.0], ['S', 60.0]]
[['O', 1.0], ['PE', 1.0]]
[['Al', 2.0], ['O', 3.0]]
[['K', 2.0], ['O', 1.0]]
[['C', 2.0], ['H', 4.0], ['O', 1.0]]
[['Er', 2.0], ['O', 6.0], ['W', 1.0]]
[['Li', 2.0], ['O', 2.0], ['O-', 1.0], ['Si', 1.0]]
[['O', 2.0], ['Zr', 1.0]]
[['O', 2.0], ['Sn', 1.0]]
[['As', 2.0], ['Ge', 1.0], ['S', 8.0], ['Sb', 2.0]]
[['Cu', 1.0], ['Ga', 1.0], ['Se', 2.0]]
[['CL', 1.0], ['Ch', 1.0]]
[['Se', 1.0], ['Te', 1.0], ['Zn', 1.0]]
[['Cu', 1.0], ['I', 2.0]]
[['O', 2.0], ['Si', 1.0]]
[['He', 1.0], ['La', 1.0]]
[['Ba', 1.0], ['O', 4.0], ['W', 1.0]]
[['Cl', 1.0], ['Cs', 1.0]]
[['As', 20.0], ['Ge', 14.0], ['Se', 66.0]]
[['Ca', 1.0], ['Dy', 1.0]]
[['N', 0.35], ['Si', 1.0]]
[['H', 2.0], ['Odipoles', 1.0]]
[['Ni', 1.0], ['O', 1.0]]
[['F', 1.0], ['NM', 1.0]]
[['O', 3.0], ['Tb', 2.0]]
[['Ox', 1.0], ['Si', 1.0]]
[['As', 1.0], ['Ga', 1.0]]
[['Al', 1.0], ['Sb', 1.0]]
[['B', 25.0], ['Ti', 10.0]]
[['Se', 1.0], ['Zn', 1.0]]
[['S', 1.0], ['Zn', 1.0]]
[['In', 1.0], ['Sb', 1.0]]
[['

In [116]:
descriptors_df = pd.merge(descriptors_df, descriptors_pub, on='normalised_name', how='outer')
descriptors_df = descriptors_df.drop_duplicates().reset_index(drop=True)
descriptors_df.to_csv('/content/descriptors_df.csv', index=False)

In [13]:
descriptors_df = pd.read_csv('descriptors_df.csv')

Еще один способ получить много дескрипторов - через библиотеку `mordred`, она выдает около 1600 дескрипторов.

In [16]:
calc = Calculator(descriptors, ignore_3D=True)

def to_mol(smile):
  mol = Chem.MolFromSmiles(smile)
  if mol:
    return mol


mols = [to_mol(smi) for smi in df_names]
df_names = 
mols = [i for i in mols if i is not None]
descriptors_mor = calc.pandas(mols)
descriptors_mor

[20:29:42] SMILES Parse Error: syntax error while parsing: [['As',
[20:29:42] SMILES Parse Error: Failed parsing SMILES '[['As',' for input: '[['As','
[20:29:42] SMILES Parse Error: syntax error while parsing: [['O',
[20:29:42] SMILES Parse Error: Failed parsing SMILES '[['O',' for input: '[['O','
[20:29:42] SMILES Parse Error: syntax error while parsing: [['Al',
[20:29:42] SMILES Parse Error: Failed parsing SMILES '[['Al',' for input: '[['Al','
[20:29:42] SMILES Parse Error: syntax error while parsing: [['K',
[20:29:42] SMILES Parse Error: Failed parsing SMILES '[['K',' for input: '[['K','
[20:29:42] SMILES Parse Error: syntax error while parsing: [['C',
[20:29:42] SMILES Parse Error: Failed parsing SMILES '[['C',' for input: '[['C','
[20:29:42] SMILES Parse Error: syntax error while parsing: [['Er',
[20:29:42] SMILES Parse Error: Failed parsing SMILES '[['Er',' for input: '[['Er','
[20:29:42] SMILES Parse Error: syntax error while parsing: [['Li',
[20:29:42] SMILES Parse Error: Faile

/usr/local/lib/python3.8/dist-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


 12%|█▏        | 152/1281 [00:30<04:05,  4.60it/s][20:30:13] WARNING: not removing hydrogen atom without neighbors
[20:30:13] WARNING: not removing hydrogen atom without neighbors
[20:30:13] WARNING: not removing hydrogen atom without neighbors
 14%|█▎        | 174/1281 [00:34<03:02,  6.07it/s][20:30:17] WARNING: not removing hydrogen atom without neighbors
[20:30:17] WARNING: not removing hydrogen atom without neighbors
[20:30:17] WARNING: not removing hydrogen atom without neighbors
 19%|█▉        | 247/1281 [00:56<24:26,  1.42s/it]

/usr/local/lib/python3.8/dist-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


 21%|██        | 265/1281 [00:58<03:33,  4.75it/s][20:30:41] WARNING: not removing hydrogen atom without neighbors
[20:30:41] WARNING: not removing hydrogen atom without neighbors
[20:30:41] WARNING: not removing hydrogen atom without neighbors
 72%|███████▏  | 922/1281 [27:11<01:24,  4.25it/s][20:56:54] WARNING: not removing hydrogen atom without neighbors
[20:56:54] WARNING: not removing hydrogen atom without neighbors
[20:56:54] WARNING: not removing hydrogen atom without neighbors
100%|██████████| 1281/1281 [28:43<00:00,  1.35s/it]


,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,2.121320,2.340100,0,0,4.472136,1.618034,3.236068,4.472136,1.118034,2.155909,...,5.509388,22.328143,58.041865,5.804186,10,1,10.0,8.0,2.5,1.250000
1,0.000000,0.000000,0,0,0.0,0.0,0.0,0.0,0.0,0.693147,...,0.000000,1.000000,28.997189,9.665730,0,0,0.0,0.0,divide by zero encountered in power (mZagreb1),0.000000
2,2.828427,3.146264,0,0,5.464102,1.732051,3.464102,5.464102,1.09282,2.390167,...,6.192362,25.583106,101.947821,20.389564,20,2,14.0,12.0,2.75,1.500000
3,4.242641,4.000000,0,0,8.0,2.0,4.0,8.0,1.333333,2.687624,...,7.627057,30.941317,138.961807,13.896181,27,3,24.0,24.0,1.5,1.500000
4,4.530370,5.004088,0,0,7.662988,2.052881,4.105762,7.662988,1.094713,2.766317,...,7.890957,32.688753,101.060255,6.316266,46,6,26.0,26.0,4.472222,1.777778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1276,6.651690,6.355891,0,0,10.79228,2.245827,4.491654,10.79228,1.199142,3.099448,...,8.806724,37.839725,233.954163,14.622135,84,10,42.0,46.0,4.083333,2.027778
1277,14.696938,14.696938,12,0,multiple fragments (SpAbs_A/SpAbs),multiple fragments (SpMax_A/SpMax),multiple fragments (SpDiam_A/SpDiam),multiple fragments (SpAD_A/SpAD),multiple fragments (SpMAD_A/SpMAD),multiple fragments (LogEE_A/LogEE),...,7.978311,57.944786,536.769646,18.509298,37000000054,0,72.0,54.0,divide by zero encountered in power (mZagreb1),6.000000
1278,3.863703,3.863703,0,0,multiple fragments (SpAbs_A/SpAbs),multiple fragments (SpMax_A/SpMax),multiple fragments (SpDiam_A/SpDiam),multiple fragments (SpAD_A/SpAD),multiple fragments (SpMAD_A/SpMAD),multiple fragments (LogEE_A/LogEE),...,6.311735,28.541146,325.866461,21.724431,1200000013,0,18.0,13.0,5.361111,2.000000
1279,8.623181,8.086503,0,0,13.878163,2.396777,4.70414,13.878163,1.261651,3.352605,...,9.317849,55.364239,151.049410,9.440588,138,14,58.0,68.0,3.694444,2.388889


In [1]:
# cols_to_use = descriptors_mor.columns.difference(descriptors_df.columns)
# df_5 = pd.merge(descriptors_df, descriptors_mor[cols_to_use], on='normalised_name', how='outer')
# df_5 = df_5.drop_duplicates().reset_index(drop=True)
# df_5
# # df_5.to_csv('/content/df_3.csv', index=False)

In [55]:
mols = [to_mol(smi) for smi in df_names]
df_names = df_names.reset_index(drop=True)
df_names = df_names[~pd.Series(mols).isna()].reset_index(drop=True)
descriptors_mor['normalised_name'] = df_names

In [56]:
descriptors_mor

,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2,normalised_name
0,2.121320,2.340100,0,0,4.472136,1.618034,3.236068,4.472136,1.118034,2.155909,...,22.328143,58.041865,5.804186,10,1,10.0,8.0,2.5,1.250000,CCC=O
1,0.000000,0.000000,0,0,0.0,0.0,0.0,0.0,0.0,0.693147,...,1.000000,28.997189,9.665730,0,0,0.0,0.0,divide by zero encountered in power (mZagreb1),0.000000,[AlH2]
2,2.828427,3.146264,0,0,5.464102,1.732051,3.464102,5.464102,1.09282,2.390167,...,25.583106,101.947821,20.389564,20,2,14.0,12.0,2.75,1.500000,O=[Al]O[Al]=O
3,4.242641,4.000000,0,0,8.0,2.0,4.0,8.0,1.333333,2.687624,...,30.941317,138.961807,13.896181,27,3,24.0,24.0,1.5,1.500000,N1PNP=NP1
4,4.530370,5.004088,0,0,7.662988,2.052881,4.105762,7.662988,1.094713,2.766317,...,32.688753,101.060255,6.316266,46,6,26.0,26.0,4.472222,1.777778,COC(=O)[C](C)C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1276,6.651690,6.355891,0,0,10.79228,2.245827,4.491654,10.79228,1.199142,3.099448,...,37.839725,233.954163,14.622135,84,10,42.0,46.0,4.083333,2.027778,CC1=CC(=C(C=C1)O)I
1277,14.696938,14.696938,12,0,multiple fragments (SpAbs_A/SpAbs),multiple fragments (SpMax_A/SpMax),multiple fragments (SpDiam_A/SpDiam),multiple fragments (SpAD_A/SpAD),multiple fragments (SpMAD_A/SpMAD),multiple fragments (LogEE_A/LogEE),...,57.944786,536.769646,18.509298,37000000054,0,72.0,54.0,divide by zero encountered in power (mZagreb1),6.000000,[Be+2].[Be+2].[Be+2].[O-][Si](=O)[O-].[O-][Si]...
1278,3.863703,3.863703,0,0,multiple fragments (SpAbs_A/SpAbs),multiple fragments (SpMax_A/SpMax),multiple fragments (SpDiam_A/SpDiam),multiple fragments (SpAD_A/SpAD),multiple fragments (SpMAD_A/SpMAD),multiple fragments (LogEE_A/LogEE),...,28.541146,325.866461,21.724431,1200000013,0,18.0,13.0,5.361111,2.000000,CC(C)=O.ICI
1279,8.623181,8.086503,0,0,13.878163,2.396777,4.70414,13.878163,1.261651,3.352605,...,55.364239,151.049410,9.440588,138,14,58.0,68.0,3.694444,2.388889,C1=NC2=C(N1)C(=O)NC(=N2)N


In [57]:
descriptors_mor.to_csv('descriptors_mor.csv', index=False)